In [69]:
import os
from dotenv import load_dotenv
from faker import Faker
from datetime import datetime, timedelta
import random
from datetime import datetime
import time, pymongo
from pymongo import *

In [70]:
# Connect to the MongoDB server
client = MongoClient('localhost', 27017)
db = client["darkstar-games"]
collection ="transactions" 
# Ensure the transactions collection is created as a time-series collection
time_series_options = {
    "timeField": "timestamp",  # Field used as timeField
    "metaField": "metadata",   # Field used as metaField
    "granularity": "minutes"   # Setting granularity to minutes
}

# Check if the collection exists, and create it if not
if collection not in db.list_collection_names():
    db.create_collection(collection, timeseries=time_series_options)
    print(f"Collection '{collection}' créée comme collection de séries temporelles.")

# Obtenir la référence de la collection et créer un index sur timestamp si nécessaire
transactions = db[collection]
transactions.create_index([("timestamp", DESCENDING)])

# Initialize Faker
fake = Faker()

# Generate a list of unique user IDs
unique_user_ids = [fake.uuid4() for _ in range(500)]

# Generate a list of unique item IDs (limited to 100)
unique_item_ids = [fake.uuid4() for _ in range(100)]

In [71]:
# Start date for timestamp
start_date = datetime(2024, 5, 1)

prev_timestamp = start_date  # Initialize previous timestamp
iterations = 0
batch_size = 100
transaction_batch=[]
ajouts = 0
# Generate transactions in real time and insert them into the database
while True:
    seller_id = random.choice(unique_user_ids)
    buyer_id = random.choice(unique_user_ids)
    item_id = random.choice(unique_item_ids)

    # Ensure that the seller and buyer are not the same
    while buyer_id == seller_id:
        buyer_id = random.choice(unique_user_ids)

    item_sale_price = round(random.uniform(10, 1000), 2)  # Sale price between 10 and 1000

    # Calculate listing fee: max(10 credits or 5% of the sale price)
    listing_fee = max(10, 0.05 * item_sale_price)

    # Calculate transaction fee: 10% of the sale price
    transaction_fee = 0.1 * item_sale_price
    total_fees = item_sale_price + listing_fee + transaction_fee

    # Generate random balances before the sale for seller and buyer
    seller_balance_before = round(random.uniform(1000, 10000), 2)
    buyer_balance_before = round(random.uniform(item_sale_price, 10000), 2)

    # Calculate seller's balance after the sale
    seller_balance_after = seller_balance_before + item_sale_price - listing_fee - transaction_fee

    # Calculate buyer's balance after the sale
    buyer_balance_after = buyer_balance_before - item_sale_price
    choices = [True, False]
    weights = [97, 3]  # Correspond aux pourcentages respectifs pour True et False

    # Introduce occasional False values in boolean flags
    boolean_left_seller_inventory = random.choices(choices, weights=weights, k=1)[0]
    boolean_entered_buyer_inventory = random.choices(choices, weights=weights, k=1)[0]
    # Generate timestamp
    timestamp = prev_timestamp + timedelta(minutes=random.randint(1, 10))  # Add random minutes to the previous timestamp

    # Introduce errors in balances with 3% probability
    if random.choices([True, False], weights=[97, 3], k=1)[0]:  # 3% probability of error
        # Simulate incorrect balances
        seller_balance_after += random.uniform(-10, 10)  # Add a small random amount
    if random.choices([True, False], weights=[97, 3], k=1)[0]:  # 3% probability of error
        buyer_balance_after += random.uniform(-10, 10)  # Add a small random amount

    # Prepare transaction data
    transaction_data = {
        "timestamp": timestamp,
        "metadata": {
            "seller_player_id": str(seller_id),
            "buyer_player_id": str(buyer_id),
            "item_id": str(item_id),
            "item_sale_price": float(item_sale_price),
            "listing_fee": float(listing_fee),
            "transaction_fee": float(transaction_fee),
            "total_fees": float(total_fees),
            "seller_balance_before": float(seller_balance_before),
            "seller_balance_after": float(seller_balance_after),
            "buyer_balance_before": float(buyer_balance_before),
            "buyer_balance_after": float(buyer_balance_after),
            "left_seller_inventory": boolean_left_seller_inventory,
            "entered_buyer_inventory": boolean_entered_buyer_inventory
        }
    }

    transaction_batch.append(transaction_data)
    iterations += 1
    prev_timestamp = timestamp  # Update previous timestamp
    if len(transaction_batch) >= batch_size:
        transactions.insert_many(transaction_batch)
        transaction_batch =[]
        ajouts += 1
    print(f"\r{' ' * 100}\r{iterations} documents created, {ajouts} documents insertions made, {count} documents in the collection", end="", flush=True)

    count = transactions.count_documents({})
    if count == 500000:
        break

    time.sleep(0.1)  # Simulate real-time transaction generation by waiting 

print("Program entirely executed")

28523 documents created, 285 documents insertions made, 82400 documents in the collection           

KeyboardInterrupt: 